## Implicit collaborative filtering approaches

### Background

This is the development pipeline to apply the implicit item-based recommendation system algorithms for gracioustyle customers. The aim is to test and compare a few algorithms including  
  
* Alternating Least Square
* Bayesian Personalized Ranking
  
The approaches above can be built into a hybrid CF approach, and further combined with the content based apporach as the optimized strategy. 

### Import libraries

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype
import random
import implicit
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import normalize

from implicit.als import AlternatingLeastSquares
from implicit.bpr import BayesianPersonalizedRanking
from implicit.datasets.movielens import get_movielens
from implicit.lmf import LogisticMatrixFactorization
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight,tfidf_weight)
import tqdm
import codecs
from tabulate import tabulate

### Import data

Order data has been merged with product table to build a master source data in json/csv format.

In [2]:
retail_data = pd.read_json('C:/Projects/DJ_projects/FP-growth/recommendation_engine/development/hybrid/data/cus_user_quality_date_0716.json') # This may take a couple minutes
# df2 = pd.read_csv('data/product_brand_type_color.csv') # This may take a couple minutes

In [3]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120442 entries, 0 to 120441
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CUSTOMER_DIM_KEY   120442 non-null  int64  
 1   PRODUCT_DIM_KEY    120442 non-null  int64  
 2   QUANTITY           120441 non-null  float64
 3   PRODUCT_NAME       120435 non-null  object 
 4   COLOR              120442 non-null  object 
 5   PRODUCT_TYPE       120442 non-null  object 
 6   CATEGORY           120442 non-null  object 
 7   BRAND              120442 non-null  object 
 8   RESOLUTION_STATUS  120442 non-null  object 
 9   ORDER_DATE_KEY     120442 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 9.2+ MB


Some data subseting has been done based on the following conditions,  
  
* Only contains the order history on products that have had orders since 2018. This is used as a proxy as the current product catelogue. 
* Resolution status in 'Completed' or 'Pending'.
* De-duplicate the identical rows.

In [4]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.PRODUCT_NAME) == False]
cleaned_retail = cleaned_retail[cleaned_retail['PRODUCT_DIM_KEY'].isin(cleaned_retail[cleaned_retail['ORDER_DATE_KEY'] > 17533].PRODUCT_DIM_KEY)]
cleaned_retail['QUANTITY'] = cleaned_retail['QUANTITY'].fillna(0.0)
cleaned_retail = cleaned_retail[cleaned_retail.RESOLUTION_STATUS.isin(['COMPLETED', 'PENDING'])]
print('Duplicated rows: ' + str(cleaned_retail.duplicated().sum()))
cleaned_retail.drop_duplicates(inplace=True)

Duplicated rows: 280


In [5]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51810 entries, 0 to 120440
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CUSTOMER_DIM_KEY   51810 non-null  int64  
 1   PRODUCT_DIM_KEY    51810 non-null  int64  
 2   QUANTITY           51810 non-null  float64
 3   PRODUCT_NAME       51810 non-null  object 
 4   COLOR              51810 non-null  object 
 5   PRODUCT_TYPE       51810 non-null  object 
 6   CATEGORY           51810 non-null  object 
 7   BRAND              51810 non-null  object 
 8   RESOLUTION_STATUS  51810 non-null  object 
 9   ORDER_DATE_KEY     51810 non-null  int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 4.3+ MB


In [6]:
cleaned_retail['PRODUCT_TYPE'].value_counts()

dinner plates                                                        2630
table protectors                                                     2602
coffee mugs & tea cups                                               1844
Unknown                                                              1698
tablecloths                                                          1628
                                                                     ... 
dinner knives,dinner forks,dinner spoons,teaspoons,place settings       1
coasters & trivets,bottle openers                                       1
napkins,napkin holders                                                  1
cocktail glasses,handle,spoons                                          1
serving trays,serving trays                                             1
Name: PRODUCT_TYPE, Length: 992, dtype: int64

### Threhold function for cut-off

In [7]:
def threshold_likes(df, uid_min, mid_min, user_column = 'CUSTOMER_DIM_KEY', item_column = 'PRODUCT_DIM_KEY'):
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby(user_column)[item_column].count()
        df = df[~df[user_column].isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby(item_column)[user_column].count()
        df = df[~df[item_column].isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby(user_column)[item_column].count().min() >= mid_min)
    assert(df.groupby(item_column)[user_column].count().min() >= uid_min)
    
    n_users = df[user_column].unique().shape[0]
    n_items = df[item_column].unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

### Product lookup table

In [8]:
df_lim = threshold_likes(cleaned_retail, 2, 2)
item_lookup = df_lim[['PRODUCT_DIM_KEY', 'PRODUCT_NAME']].drop_duplicates() # Only get unique item/description pairs
#item_lookup['PRODUCT_DIM_KEY'] = item_lookup.PRODUCT_DIM_KEY.astype(str) # Encode as strings for future lookup ease

Starting likes info
Number of users: 27143
Number of models: 17314
Sparsity: 0.011%
Ending likes info
Number of users: 7088
Number of models: 5368
Sparsity: 0.064%


### User-product matrix function

In [9]:
def create_matrix(cleaned_retail, row_feature, col_feature):
    cleaned_retail[row_feature] = cleaned_retail[row_feature].astype(int) # Convert to int for customer ID
    cleaned_retail = cleaned_retail[[col_feature, 'QUANTITY', row_feature]] # Get rid of unnecessary info
    grouped_cleaned = cleaned_retail.groupby([row_feature, col_feature]).sum().reset_index() # Group together
    grouped_cleaned.QUANTITY.loc[grouped_cleaned.QUANTITY == 0] = 1 # Replace a sum of zero purchases with a one to
    # indicate purchased
    grouped_purchased = grouped_cleaned.query('QUANTITY > 0') # Only get customers where purchase totals were positive
    
    customers = list(np.sort(grouped_purchased[row_feature].unique())) # Get our unique customers
    products = list(np.sort(grouped_purchased[col_feature].unique())) # Get our unique products that were purchased
    quantity = list(grouped_purchased.QUANTITY) # All of our purchases
    #print(products)
    rows = grouped_purchased[row_feature].astype('category', CategoricalDtype(categories = customers)).cat.codes 
    # Get the associated row indices
    cols = grouped_purchased[col_feature].astype('category', CategoricalDtype(categories = products)).cat.codes 
    # Get the associated column indices
    purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))
    return purchases_sparse, customers, products

### Create user-product matrix

In [10]:
purchases_sparse, products, customers = create_matrix(df_lim, 'PRODUCT_DIM_KEY', 'CUSTOMER_DIM_KEY')

C:\Users\david_qi\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [11]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [12]:
customers_arr

array([31785, 31786, 31795, ..., 83482, 83502, 83509], dtype=int64)

In [13]:
len(set(customers_arr))

7088

### Train and test data prep

In [15]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered

In [16]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

### Function to calculate AUC (evaluation metric)

In [17]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr) 

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
def calc_mean_auc_cv(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
def calc_mean_auc_pop(training_set, altered_users, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    
    return float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'regularization': [0.0, 1e-5, 1e-3, 1e-1],
              'alpha': [1, 10, 50, 100]}

def cross_validation_als(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    #product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)
    #product_train, product_test, product_users_altered = make_train(product_train, pct_test = 0.2)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9)).tocsr()

    model = AlternatingLeastSquares()
    pop_auc = calc_mean_auc_pop(product_train, product_users_altered, product_test)

    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_model = model
    print('The mean AUC of popularity benchmark: ', pop_auc)
    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for alpha in param_grid['alpha']:
                model = AlternatingLeastSquares(factors = factor, regularization = regu)
                model.fit(training_set*alpha)
                item_vecs = model.user_factors
                user_vecs = model.item_factors
                store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                print(factor, regu, alpha, store_auc)
                
                if store_auc > best_auc:
                    best_auc = store_auc
                    best_factors = factor
                    best_reg = regu
                    best_alpha = alpha
                    best_model = model
                    
    print('Best AUC: ', best_auc)
    print('Pop AUC: ', pop_auc)
    print('Best factor: ', best_factors)
    print('Best regularization: ', best_reg)
    print('Best alpha: ', best_alpha)
    return best_auc, pop_auc, best_factors, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

### ALS hyper tuning with cross validation

In [29]:
cross_validation_als(purchases_sparse, 'als', param_grid)

The mean AUC of popularity benchmark:  0.674



10 0.1 1 0.748


KeyboardInterrupt: 

### ALS best fitting

In [19]:
model = AlternatingLeastSquares(factors=80,regularization=0.0001)
# The literature suggests values between 0 and 3 for K1 and 0 and 1 for B. 
ratings = bm25_weight(product_train, K1= 0.1, B=1).tocsr()
model.fit(ratings)
item_vecs = model.user_factors
user_vecs = model.item_factors
store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
store_auc, pop_auc

(0.774, 0.674)

In [33]:
param_grid = {'B': [0.1, 0.5, 1, 2, 3],
              'K1': [0, 0.1, 0.01, 1]
             }

def cross_validation_als_bm25_param(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    #product_train, product_out_test, product_users_out_altered = make_train(purchases_sparse, pct_test = 0.2)
    #product_train, product_test, product_users_altered = make_train(product_train, pct_test = 0.2)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    training_set = (bm25_weight(product_train, B=0.9)).tocsr()

    model = AlternatingLeastSquares()
    pop_auc = calc_mean_auc_pop(product_train, product_users_altered, product_test)
    best_auc=0
    best_B = 0
    best_K1 = 0
    best_model = model
    print('The mean AUC of popularity benchmark: ', pop_auc)
    for B in param_grid['B']:
        for K1 in param_grid['K1']:
            training_set = (bm25_weight(product_train, B=B,K1=K1)).tocsr()
            model = AlternatingLeastSquares(factors = 80, regularization = 0.0001)
            model.fit(training_set)
            item_vecs = model.user_factors
            user_vecs = model.item_factors
            store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
            print(B, K1, store_auc)
                
            if store_auc > best_auc:
                best_auc = store_auc
                best_B = B
                best_K1 = K1
                best_model = model
                    
    print('Best AUC: ', best_auc)
    print('Pop AUC: ', pop_auc)
    print('Best B: ', best_B)
    print('Best K1: ', best_K1)
    return best_auc, pop_auc, best_B, best_K1, best_model
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [34]:
cross_validation_als_bm25_param(purchases_sparse, 'als', param_grid)

The mean AUC of popularity benchmark:  0.674



0.1 0 0.766



0.1 0.1 0.775



0.1 0.01 0.765



0.1 1 0.776


...


3 0 0.772



3 0.1 0.783



3 0.01 0.773



3 1 0.669
Best AUC:  0.796
Pop AUC:  0.674
Best B:  1
Best K1:  1


(0.796, 0.674, 1, 1, <implicit.als.AlternatingLeastSquares at 0x2369c2bf808>)

### BPR hyper tuning with cross validation

In [39]:
'''
param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [1, 0.1, 0.01, 0.001]
              'regularization': [0.0, 0.1,0.01,0.001,0.0001,10],
              'alpha': [1, 10, 50, 100, 500, 1000]}
'''

param_grid = {'num_factors': [10, 20, 40, 80, 120],
              'learning_rate': [0.1],
              'regularization': [0.0, 0.1,0.01,0.001],
              'alpha': [1, 10, 50, 100, 500]}

def cross_validation_bpr(sparse_matrix, model_name, param_grid, pct_test=0.2):
    product_train, product_test, product_users_altered = make_train(sparse_matrix, pct_test = pct_test)
    # item_vecs, user_vecs, model = train_model(training_set, model_name)
    if model_name == "bpr":
        model = BayesianPersonalizedRanking()
    if model_name == "lmf":
        model = LogisticMatrixFactorization()
        
    training_set = product_train
    best_auc = 0
    best_factors = 0
    best_reg = 0
    best_alpha = 0
    best_learning_rate = 0

    for factor in param_grid['num_factors']:
        for regu in param_grid['regularization']:
            for lr in param_grid['learning_rate']:
                for alpha in param_grid['alpha']:
                    model = BayesianPersonalizedRanking(factors = factor,learning_rate= lr, regularization = regu)
                    model.fit(training_set*alpha)
                    item_vecs = model.user_factors
                    user_vecs = model.item_factors
                    store_auc = calc_mean_auc_cv(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
                    print(factor, regu, lr, alpha, store_auc, pop_auc)

                    if store_auc > best_auc:
                        best_auc = store_auc
                        best_factors = factor
                        best_reg = regu
                        best_alpha = alpha
                        best_learning_rate = lr
    print(best_auc, best_factors, best_learning_rate, best_reg, best_alpha)
    return best_auc, best_factors,best_learning_rate, best_reg, best_alpha
    # print("trained model '%s' in %s", model_name, time.time() - start)

In [40]:
cross_validation_bpr(purchases_sparse, 'bpr', param_grid)


10 0.0 0.1 1 0.749 0.674



10 0.0 0.1 10 0.759 0.674



10 0.0 0.1 50 0.757 0.674



10 0.0 0.1 100 0.755 0.674



10 0.0 0.1 500 0.759 0.674


...


80 0.001 0.1 50 0.785 0.674



80 0.001 0.1 100 0.788 0.674



80 0.001 0.1 500 0.789 0.674



120 0.0 0.1 1 0.787 0.674


KeyboardInterrupt: 

In [27]:
model = BayesianPersonalizedRanking(factors=40,regularization=0.1,learning_rate=0.1)
# The literature suggests values between 0 and 3 for K1 and 0 and 1 for B. 
ratings = bm25_weight(product_train, K1= 1, B=0.9).tocsr()
model.fit(product_train*50)
item_vecs = model.user_factors
user_vecs = model.item_factors
store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
store_auc, pop_auc

(0.801, 0.674)

In [114]:
customers_arr[5000:5100]

array([71809, 71818, 71819, 71825, 71829, 71844, 71845, 71846, 71860,
       71865, 71872, 71884, 71886, 71887, 71907, 71908, 71909, 71911,
       71914, 71924, 71927, 71928, 71929, 71934, 71937, 71942, 71954,
       71955, 71958, 71963, 71965, 71967, 71969, 71976, 71983, 71985,
       71988, 71989, 71995, 72013, 72016, 72017, 72018, 72026, 72027,
       72034, 72037, 72046, 72050, 72052, 72054, 72055, 72059, 72061,
       72065, 72067, 72069, 72071, 72077, 72082, 72094, 72095, 72100,
       72101, 72108, 72112, 72114, 72118, 72120, 72143, 72145, 72154,
       72160, 72163, 72164, 72165, 72169, 72172, 72174, 72175, 72180,
       72200, 72202, 72210, 72211, 72212, 72217, 72221, 72245, 72254,
       72281, 72287, 72289, 72290, 72300, 72306, 72307, 72310, 72312,
       72313], dtype=int64)

### Get actual purchase vs model recommendations

In [28]:
def get_rec_items(model, customer_id, sparse_matrix, customer_list, item_list, item_lookup, num_items = 10):
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = sparse_matrix[:,cust_ind].nonzero()[0] # Get column indices of purchased items
    prod_codes = item_list[purchased_ind] # Get the stock codes for our purchased items
    print(prod_codes)
    purchased_items = item_lookup.loc[item_lookup.PRODUCT_DIM_KEY.isin(prod_codes)]
    
    print('-------------------------------------------------------------------------------------')
    print('Print purchased items in train set')
    print('-------------------------------------------------------------------------------------')
    print(tabulate(purchased_items))
    print('-------------------------------------------------------------------------------------')
    print('Print full actual purchases above threshold')
    print('-------------------------------------------------------------------------------------')
    print(tabulate(df_lim[df_lim['CUSTOMER_DIM_KEY'] == customer_id]))
    print('-------------------------------------------------------------------------------------')
    print('Print model recommendations')
    print('-------------------------------------------------------------------------------------')
    
    recom = model.recommend(cust_ind, sparse_matrix.T.tocsr(),filter_items=purchased_ind.tolist(), N= num_items)
    rec_keys = [] # start empty list to store items
    descriptions = []
    for index in recom:
        key = item_list[index[0]]
        rec_keys.append(key)
        descriptions.append(item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == key].iloc[0])
        
    scores = [item[1] for item in recom]
    final_frame = pd.DataFrame({'PRODUCT_DIM_KEY': rec_keys, 'PRODUCT_NAME': descriptions, 'SCORE': scores}) # Create a dataframe 
    final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME','SCORE']]
    return final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME','SCORE']] # Switch order of columns around

In [29]:
get_rec_items(model, 55974, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

[126746 126792]
-------------------------------------------------------------------------------------
Print purchased items in train set
-------------------------------------------------------------------------------------
-----  ------  -----------------------------------------------------------------------
10066  126746  Bouquet of Flowers Bread & Butter Plate 6.25 in Round
10943  126792  Old Master Tulips Red, Yellow, & Orange Tulip Salad Plate 7.75 in Round
-----  ------  -----------------------------------------------------------------------
-------------------------------------------------------------------------------------
Print full actual purchases above threshold
-------------------------------------------------------------------------------------
-----  -----  ------  -  -----------------------------------------------------------------------  -------  ----------------------  --------------------  ---------------  ---------  -----
10066  55974  126746  3  Bouquet of Flowers 

,PRODUCT_DIM_KEY,PRODUCT_NAME,SCORE
0,303920,Pomegranate Canister Medium,0.882262
1,506459,"Quotidien White Truffle 4pc Setting (KQ01/17, ...",0.852599
2,571974,Clothespin Pull,0.843237
3,556980,Santorini Assorted Pasta Bowls - Set Of 4 - 9....,0.830494
4,303918,Pomegranate Canister Large,0.824866
5,172921,Black Orchid Salt and Pepper,0.824067
6,522296,Kaleidoscope Illusion Purple 14x20 in. Placemat,0.823866
7,331582,Tortoise Salad Fork,0.800035
8,183142,Aladdin Brilliant Clear Five-Piece Place Setting,0.756955
9,405776,Jagger Arch Mirror Antique Gold,0.731300


In [70]:
model = AlternatingLeastSquares(factors=80,regularization=0.0001)
# The literature suggests values between 0 and 3 for K1 and 0 and 1 for B. 
ratings = bm25_weight(product_train, K1= 0.1, B=1).tocsr()
model.fit(ratings)
item_vecs = model.user_factors
user_vecs = model.item_factors
store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
store_auc, pop_auc

(0.775, 0.674)

In [26]:
get_rec_items(model, 55974, product_test, customers_arr, products_arr, item_lookup, num_items = 20)

[126746 126792]
-------------------------------------------------------------------------------------
Print purchased items in train set
-------------------------------------------------------------------------------------
-----  ------  -----------------------------------------------------------------------
10066  126746  Bouquet of Flowers Bread & Butter Plate 6.25 in Round
10943  126792  Old Master Tulips Red, Yellow, & Orange Tulip Salad Plate 7.75 in Round
-----  ------  -----------------------------------------------------------------------
-------------------------------------------------------------------------------------
Print full actual purchases above threshold
-------------------------------------------------------------------------------------
-----  -----  ------  -  -----------------------------------------------------------------------  -------  ----------------------  --------------------  ---------------  ---------  -----
10066  55974  126746  3  Bouquet of Flowers 

,PRODUCT_DIM_KEY,PRODUCT_NAME,SCORE
0,303920,Pomegranate Canister Medium,0.056038
1,303918,Pomegranate Canister Large,0.047937
2,172958,Botanical Leaf Tissue Box,0.044049
3,172993,Black Orchid Candle,0.040760
4,172921,Black Orchid Salt and Pepper,0.029125
5,124043,Linen Wash Original Floral Fusion 64 oz,0.027242
6,171325,Vuelta Atoll/Ocean Blue Cake Dish/Tray 12.6 x ...,0.026932
7,171304,Vuelta Atoll/Ocean Blue individual Bowl 6.3 in,0.026759
8,348315,Napali Set Of 2: Desk Blotter And Round Mouse ...,0.026302
9,187250,Straw Loop Natural Placemat,0.024801


In [72]:
def get_sim_item(item_id):
    purchased_items = item_lookup.loc[item_lookup.PRODUCT_DIM_KEY.isin([item_id])]
    item_ind = np.where(products_arr == item_id)
    print(purchased_items)
    arr = model.similar_items(item_ind[0][0], N=20)
    rec_keys = [] # start empty list to store items
    descriptions = []
    for index in arr:
        key = products_arr[index[0]]
        rec_keys.append(key)
        descriptions.append(item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == key].iloc[0])
        
    scores = [item[1] for item in arr]
    final_frame = pd.DataFrame({'PRODUCT_DIM_KEY': rec_keys, 'PRODUCT_NAME': descriptions, 'SCORE': scores}) # Create a dataframe 
    final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME','SCORE']]
    return final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME','SCORE']] # Switch order of columns around

In [75]:
#BPR
get_sim_item(126746)

       PRODUCT_DIM_KEY                                       PRODUCT_NAME
10066           126746  Bouquet of Flowers Bread & Butter Plate 6.25 i...


,PRODUCT_DIM_KEY,PRODUCT_NAME,SCORE
0,126746,Bouquet of Flowers Bread & Butter Plate 6.25 i...,1.089021
1,303920,Pomegranate Canister Medium,0.976083
2,303918,Pomegranate Canister Large,0.912312
3,172921,Black Orchid Salt and Pepper,0.847773
4,172958,Botanical Leaf Tissue Box,0.690134
5,172993,Black Orchid Candle,0.596023
6,499116,Vecchio Ginori 9 3/4 In. Salad Bowl Vecchio Gi...,0.574746
7,471592,Christmas Tartan Red Napkin,0.559289
8,403632,L'Ame Stainless Steel Dessert/Salad Fork,0.533346
9,304320,Bona Acrylic Small Bottle Clear,0.523527


In [73]:
#ALS
get_sim_item(126746)

       PRODUCT_DIM_KEY                                       PRODUCT_NAME
10066           126746  Bouquet of Flowers Bread & Butter Plate 6.25 i...


,PRODUCT_DIM_KEY,PRODUCT_NAME,SCORE
0,126746,Bouquet of Flowers Bread & Butter Plate 6.25 i...,0.026698
1,172921,Black Orchid Salt and Pepper,0.026629
2,303918,Pomegranate Canister Large,0.026397
3,303920,Pomegranate Canister Medium,0.026338
4,172993,Black Orchid Candle,0.026225
5,172958,Botanical Leaf Tissue Box,0.025877
6,109334,Majestic Forest Pewter Acorn Salt And Pepper Set,0.025873
7,148458,"Sailboat, Medium",0.025836
8,148459,"Sailboat, Small",0.025526
9,336051,Bumblebee Candle,0.025458


In [44]:
def get_rec_items_model(customer_id, sparse_matrix, customer_list, item_list, item_lookup, num_items = 10, model_name='als'):
    if model_name == 'als':
        model = AlternatingLeastSquares(factors=80,regularization=0.0001)
    elif model_name == 'bpr':
        model = BayesianPersonalizedRanking(factors=40,regularization=0.1,learning_rate=0.1)
    else:
        print('Wrong model name')
        return
    # The literature suggests values between 0 and 3 for K1 and 0 and 1 for B. 
    ratings = bm25_weight(product_train, K1= 1, B=0.9).tocsr()
    model.fit(product_train*50)
    item_vecs = model.user_factors
    user_vecs = model.item_factors
    store_auc, pop_auc = calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
    store_auc, pop_auc
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = sparse_matrix[:,cust_ind].nonzero()[0] # Get column indices of purchased items
    prod_codes = item_list[purchased_ind] # Get the stock codes for our purchased items
    print(prod_codes)
    purchased_items = item_lookup.loc[item_lookup.PRODUCT_DIM_KEY.isin(prod_codes)]
    
    print('-------------------------------------------------------------------------------------')
    print('Print purchased items in train set')
    print('-------------------------------------------------------------------------------------')
    print(tabulate(purchased_items))
    print('-------------------------------------------------------------------------------------')
    print('Print full actual purchases above threshold')
    print('-------------------------------------------------------------------------------------')
    print(tabulate(df_lim[df_lim['CUSTOMER_DIM_KEY'] == customer_id]))
    print('-------------------------------------------------------------------------------------')
    print('Print model recommendations')
    print('-------------------------------------------------------------------------------------')
    
    recom = model.recommend(cust_ind, sparse_matrix.T.tocsr(),filter_items=purchased_ind.tolist(), N= num_items)
    rec_keys = [] # start empty list to store items
    descriptions = []
    for index in recom:
        key = item_list[index[0]]
        rec_keys.append(key)
        descriptions.append(item_lookup.PRODUCT_NAME.loc[item_lookup.PRODUCT_DIM_KEY == key].iloc[0])
        
    scores = [item[1] for item in recom]
    final_frame = pd.DataFrame({'PRODUCT_DIM_KEY': rec_keys, 'PRODUCT_NAME': descriptions, 'SCORE': scores}) # Create a dataframe 
    final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME','SCORE']]
    return final_frame[['PRODUCT_DIM_KEY', 'PRODUCT_NAME','SCORE']] # Switch order of columns around

In [46]:
get_rec_items_model(55974, product_test, customers_arr, products_arr, item_lookup, num_items = 20, model_name='bpr')


[126746 126792]
-------------------------------------------------------------------------------------
Print purchased items in train set
-------------------------------------------------------------------------------------
-----  ------  -----------------------------------------------------------------------
10066  126746  Bouquet of Flowers Bread & Butter Plate 6.25 in Round
10943  126792  Old Master Tulips Red, Yellow, & Orange Tulip Salad Plate 7.75 in Round
-----  ------  -----------------------------------------------------------------------
-------------------------------------------------------------------------------------
Print full actual purchases above threshold
-------------------------------------------------------------------------------------
-----  -----  ------  -  -----------------------------------------------------------------------  -------  ----------------------  --------------------  ---------------  ---------  -----
10066  55974  126746  3  Bouquet of Flowers

,PRODUCT_DIM_KEY,PRODUCT_NAME,SCORE
0,303920,Pomegranate Canister Medium,1.034620
1,303918,Pomegranate Canister Large,0.904881
2,172921,Black Orchid Salt and Pepper,0.844572
3,144205,Tattersall Black/Ecru Cotton Woven 2X3,0.806483
4,566730,Tartan Stainless Steel 24-Pc Setting (6X: Dinn...,0.785877
5,172993,Black Orchid Candle,0.760418
6,466527,Alentejo Terracotta 4.75 in Spoon Rest,0.759796
7,383531,Morning Hunt Pewter Retriever Salt And Pepper Set,0.756801
8,522523,Nixon Console,0.746710
9,462281,Elene Cream Rectangular Placemat Glass Beads,0.746562
